# Initial Import

In [1]:
from tqdm import tqdm
import os
import numpy as np
import torch
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split

from main import backgammon

/Applications/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


pygame 2.5.2 (SDL 2.28.3, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Import Dataset

In [8]:
class BackgammonDataset(Dataset):
    def __init__(self, file_path):
        # Load CSV file
        data = pd.read_csv(file_path, header=None)  # No header in dataset

        # Convert to numpy arrays
        self.X = data.iloc[:, :-1].values.astype(np.float32)  # First 28 columns as input
        self.y = data.iloc[:, -1].values.astype(np.float32)   # Last column as output

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx]), torch.tensor(self.y[idx])

# 2️⃣ Split Dataset into Training (80%) and Testing (20%)
def get_dataloaders(file_path, batch_size=32, test_size=0.2):
    dataset = BackgammonDataset(file_path)

    # Split indices into training and testing
    train_size = int((1 - test_size) * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

In [9]:
class BackgammonNet(nn.Module):
    def __init__(self):
        super(BackgammonNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(28, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

In [10]:
def get_dataloaders(file_path, batch_size=32, test_size=0.2):
    dataset = BackgammonDataset(file_path)

    # Split indices into training and testing
    train_size = int((1 - test_size) * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader


In [7]:
from time import sleep
# 3️⃣ Define Neural Network Model
class BackgammonNet(nn.Module):
    def __init__(self):
        super(BackgammonNet, self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(28, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            # nn.Dropout(0.3),
            
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            # nn.Dropout(0.3),
            
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

# 4️⃣ Train the Model
def train_model(train_loader, epochs=50, lr=0.001):
    model = BackgammonNet()
    criterion = nn.MSELoss()  # Mean Squared Error for regression
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Evaluate on test data
        model.eval()
        test_loss = 0
        # with torch.no_grad():
        #     for inputs, targets in test_loader:
        #         outputs = model(inputs)
        #         loss = criterion(outputs.squeeze(), targets)
        #         test_loss += loss.item()
        if epoch % 10 == 9:
            print(f"Epoch {epoch+1}/{epochs}, Train Loss: {total_loss:.4f}, Test Loss: {test_loss:.4f}")
            torch.save(model.state_dict(), f"backgammon_model_{epoch}.pth")
            # sleep(0.1)
            fitness = backgammon(25, "DEEP",epoch)
            print(fitness)
    return model

In [17]:
# Each point in range <= -5 - 5<=
# White and Black bar 0, 1, 2+
# Calculate blot exposure for each point 
# Calc % of home points occupied 
# Calc % of opp home points occupied
# Calc % of pieces in opp home 
# Prime present ?
# Probability blockade can be passed
class CustomDataset(Dataset):
    
    def __init__(self, input_file, output_file):
        # Load data using pandas
        self.inputs = pd.read_csv(input_file)
        self.outputs = pd.read_csv(output_file)
        
        # Ensure outputs match inputs
        assert len(self.inputs) == len(self.outputs), "Mismatched input and output lengths"
        
        # Convert to tensors
        self.inputs = torch.tensor(self.inputs.values, dtype=torch.float32)
        self.outputs = torch.tensor(self.outputs.values, dtype=torch.float32)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.outputs[idx]

# Example usage
def get_dataloader(input_file, output_file, batch_size=32, shuffle=True):
    dataset = CustomDataset(input_file, output_file)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

In [19]:
class ConnectionistModel():
    def __init__(self):
        super(ConnectionistModel, self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(289, 12),
            nn.ReLU(),
            nn.Linear(12, 1)
        )

    def forward(self, x):
        return self.model(x)

In [23]:
def train_model(train_loader, epochs=50, lr=0.001):
    model = ConnectionistModel()
    criterion = nn.MSELoss()  # Mean Squared Error for regression
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Evaluate on test data
        model.eval()
        test_loss = 0
        # with torch.no_grad():
        #     for inputs, targets in test_loader:
        #         outputs = model(inputs)
        #         loss = criterion(outputs.squeeze(), targets)
        #         test_loss += loss.item()
        if epoch % 10 == 9:
            print(f"Epoch {epoch+1}/{epochs}, Train Loss: {total_loss:.4f}, Test Loss: {test_loss:.4f}")
            torch.save(model.state_dict(), f"backgammon_model_{epoch}.pth")
            # sleep(0.1)
            # fitness = backgammon(25, "DEEP",epoch)
            # print(fitness)
    return model

In [24]:
if __name__ == "__main__":
    file_path = "../Data/Deep/BoardEquity/board_equity_db.txt"  # Update with actual dataset file
    # train_loader, test_loader = get_dataloaders(file_path, batch_size=64)
    # trained_model = train_model(train_loader, test_loader, epochs=100)
    
    train_loader = get_dataloader("../Data/Deep/289-input-x.txt", "../Data/Deep/289-input-y.txt", batch_size=32, shuffle=True)
    trained_model = train_model(train_loader, 50, 0.001)
    # Save the trained model
    torch.save(trained_model.state_dict(), "backgammon_model.pth")
    print("Model training complete and saved.")

AttributeError: 'ConnectionistModel' object has no attribute 'parameters'

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import torch.nn as nn
import torch.optim as optim

class CustomDataset(Dataset):
    def __init__(self, input_file, output_file):
        # Load data using pandas
        self.inputs = pd.read_csv(input_file)
        self.outputs = pd.read_csv(output_file)
        
        # Ensure outputs match inputs
        assert len(self.inputs) == len(self.outputs), "Mismatched input and output lengths"
        
        # Convert to tensors
        self.inputs = torch.tensor(self.inputs.values, dtype=torch.float32)
        self.outputs = torch.tensor(self.outputs.values, dtype=torch.float32)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.outputs[idx]

# Example usage
def get_dataloaders(input_file, output_file, batch_size=32, shuffle=True):
    dataset = CustomDataset(input_file, output_file)
    
    # Split dataset into 80% training and 20% test set
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, test_loader

# Define the model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(289, 1)
        # self.relu = nn.ReLU()
        # self.fc2 = nn.Linear(12, 12)
        # self.fc3 = nn.Linear(12, 1)

    def forward(self, x):
        x = self.fc1(x)
        # x = self.relu(x)
        # x = self.fc2(x)
        # x = self.relu(x)
        # x = self.fc3(x)
        return x

# Training loop
def train_model(input_file, output_file, epochs=50, batch_size=32, learning_rate=0.002):
    train_loader, test_loader = get_dataloaders(input_file, output_file, batch_size)
    model = SimpleModel()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        total_loss = 0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")
    
        # Evaluate on test set
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                test_loss += loss.item()
                
            torch.save(model.state_dict(), f"backgammon_model_{epoch}.pth")
            fitness = backgammon(25, "DEEP",epoch)
            print(fitness)
        print(f"Test Loss: {test_loss/len(test_loader):.4f}")
        
    return model
# Example usage:
model = train_model("../Data/Deep/289-input-x.txt", "../Data/Deep/289-input-y.txt")
torch.save(model.state_dict(), "backgammon_model.pth")
print("Model training complete and saved.")


Epoch 1/50, Loss: 0.3033
([0, 0, 0], 0, [0, 0, 9], 27)
Test Loss: 0.2104
Epoch 2/50, Loss: 0.1831
([0, 0, 0], 0, [0, 0, 9], 27)
Test Loss: 0.1574
Epoch 3/50, Loss: 0.1464
([0, 0, 0], 0, [0, 0, 9], 27)
Test Loss: 0.1346
Epoch 4/50, Loss: 0.1289
([0, 0, 0], 0, [0, 0, 9], 27)
Test Loss: 0.1273
Epoch 5/50, Loss: 0.1176
([0, 0, 0], 0, [0, 0, 9], 27)
Test Loss: 0.1219
Epoch 6/50, Loss: 0.1109
([0, 0, 0], 0, [0, 0, 9], 27)
Test Loss: 0.1130
Epoch 7/50, Loss: 0.1060
([0, 0, 0], 0, [0, 0, 9], 27)
Test Loss: 0.1095
Epoch 8/50, Loss: 0.1023
([0, 0, 0], 0, [0, 0, 9], 27)
Test Loss: 0.1094
Epoch 9/50, Loss: 0.0982
([0, 0, 0], 0, [0, 0, 9], 27)
Test Loss: 0.1084
Epoch 10/50, Loss: 0.0963
([0, 0, 0], 0, [0, 0, 9], 27)
Test Loss: 0.1034
Epoch 11/50, Loss: 0.0940
([0, 0, 0], 0, [0, 0, 9], 27)
Test Loss: 0.1014
Epoch 12/50, Loss: 0.0911
([0, 0, 0], 0, [0, 0, 9], 27)
Test Loss: 0.0998
Epoch 13/50, Loss: 0.0896
([0, 0, 0], 0, [0, 1, 8], 26)
Test Loss: 0.1014
Epoch 14/50, Loss: 0.0896
([0, 0, 0], 0, [0, 0,

RuntimeError: Parent directory /models does not exist.